In [2]:
import numpy as np
import pandas as pd
import random
import time
import scipy as sp
from sklearn.metrics.pairwise import rbf_kernel
from scipy.sparse.linalg import cg

In [3]:
from data_datasets import higgs, susy, cod_rna
N = 3000
data = "susy"
# data = "higgs"
# data = "cod_rna"
####################

if data == "higgs":
    X_train, X_test, y_train, y_test = higgs(N)
elif data == "susy":
    X_train, X_test, y_train, y_test = susy(N)
elif data == "cod_rna":
    X_train, X_test, y_train, y_test = cod_rna(N)

print("\nDataset:", data)
print("--------\nShape train data:", X_train.shape)
print("Shape test data:", X_test.shape)


Dataset: susy
--------
Shape train data: (3000, 18)
Shape test data: (3000, 18)


In [ ]:

import numpy as np
from scipy.sparse.linalg import cg
# import tensorflow as tf
import time


def conjugate_grad(A, b, x=None):
    """
    Description
    -----------
    Solve a linear equation Ax = b with conjugate gradient method.
    Parameters
    ----------
    A: 2d numpy.array of positive semi-definite (symmetric) matrix
    b: 1d numpy.array
    x: 1d numpy.array of initial point
    Returns
    -------
    1d numpy.array x such that Ax = b
    """
    n = len(b)
    if not x:
        x = np.ones(n)
    r = np.dot(A, x) - b
    p = - r
    r_k_norm = np.dot(r, r)
    for i in range(2*n):
        Ap = np.dot(A, p)
        alpha = r_k_norm / np.dot(p, Ap)
        x += alpha * p
        r += alpha * Ap
        r_kplus1_norm = np.dot(r, r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        if r_kplus1_norm < 1e-5:
            print ('Itr:', i)
            break
        p = beta * p - r
    return x

# x_ph = tf.placeholder('float32', [None, None])
# r = tf.matmul(A, x_ph) - b

In [5]:
K = rbf_kernel(X_train, gamma=.1)
K

array([[1.        , 0.00257308, 0.00449436, ..., 0.00368384, 0.004638  ,
        0.00114525],
       [0.00257308, 1.        , 0.46284024, ..., 0.09177461, 0.03373803,
        0.28276524],
       [0.00449436, 0.46284024, 1.        , ..., 0.2963664 , 0.10127718,
        0.30640716],
       ...,
       [0.00368384, 0.09177461, 0.2963664 , ..., 1.        , 0.07024007,
        0.07830995],
       [0.004638  , 0.03373803, 0.10127718, ..., 0.07024007, 1.        ,
        0.04738602],
       [0.00114525, 0.28276524, 0.30640716, ..., 0.07830995, 0.04738602,
        1.        ]])

In [6]:
from low_rank_methods import nystrom
U,D = nystrom(K,20)
mu=.02

Y_shift.shape (3000, 20)
C.shape (20, 20)
B.shape (3000, 20)
U.shape (3000, 20)
D.shape (20,)
VT.shape (20, 20)
Sigma.shape (20, 20)
Lamda.shape (20, 20) [[469.23277663   0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.          82.55311324   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.           0.          64.20876012   0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.          51.59337157   0.
    0.           0.           0.           0.       

In [7]:
K_reg=U@U.T+mu*np.eye(U.shape[0])
K_reg

array([[ 0.02314162,  0.00164071, -0.0001143 , ..., -0.00041997,
        -0.00033558, -0.00111613],
       [ 0.00164071,  0.02615047,  0.00243347, ..., -0.00210378,
        -0.00062976,  0.00302964],
       [-0.0001143 ,  0.00243347,  0.0276363 , ...,  0.00119334,
        -0.00246457,  0.00321178],
       ...,
       [-0.00041997, -0.00210378,  0.00119334, ...,  0.02882129,
        -0.00038525, -0.00121669],
       [-0.00033558, -0.00062976, -0.00246457, ..., -0.00038525,
         0.02593216, -0.00087452],
       [-0.00111613,  0.00302964,  0.00321178, ..., -0.00121669,
        -0.00087452,  0.02586542]])

In [8]:
def efficient_woodbury(A, U, V, k, C=None):
    '''
    Computes the Woodbury identity for the matrix A + UCV^T where A is a diagonal matrix.
    '''

    A_inv_diag = 1./np.diag(A)  # note! A_inv_diag is a vector!
    if C is None:
        B_inv = np.linalg.solve(np.eye(k) + (V * A_inv_diag) @ U, V)
    else:
        B_inv = np.linalg.solve(np.linalg.inv(C) + (V * A_inv_diag) @ U, V)

    return np.diag(A_inv_diag) - (A_inv_diag.reshape(-1, 1) * U @ B_inv * A_inv_diag)

In [11]:
N, d = U.shape
i = np.eye(d)
I = np.eye(N)

K_reg_inv=efficient_woodbury(mu*I, U, U.T, 20)

In [12]:
np.linalg.cond(K_reg_inv@K_reg)

1.0000000000000215